In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

#============================================================== Recursive Coin Population =================================================================================

def populate_coins(coins, R, coin_radius, max_depth = 5000, depth=0):
    if depth > max_depth:
        return None

    r = (R-coin_radius) * np.sqrt(np.random.random())
    theta = 2 * np.pi * np.random.random()
    sx = r * np.cos(theta)
    sy = r * np.sin(theta)

    if np.sqrt(sx**2 + sy**2) <= coin_radius * 2:
        return populate_coins(coins, R, coin_radius, max_depth, depth + 1)

    for (ex, ey) in coins:
        dist = np.sqrt((sx - ex)**2 + (sy - ey)**2)
        if dist < 2 * coin_radius:
            return populate_coins(coins, R, coin_radius, max_depth, depth + 1)

    return (sx, sy)

#============================================================== Scatter and Reflection functions ==========================================================================

def scatter():
    direction = 2 * np.pi * np.random.random()
    dx, dy = np.cos(direction), np.sin(direction)
    return dx, dy

def reflect(R, x, y):
    dx, dy = scatter()
    rx, ry = x/(R), y/(R)
    v_radial = dx*rx + dy*ry
        
    if v_radial > 0:
        dx -= 2*v_radial*rx
        dy -= 2*v_radial*ry

    return dx,dy

def discriminant(x,y,dx,dy,R):
    b = 2*(x*dx + y*dy)
    c = x**2 + y**2 - R**2
    discriminant = b**2 - 4*c
    return b,discriminant

#============================================================== Particle Simulations ======================================================================================

def simulate_particle_to_first_collision(R, n, coin_radius, v, max_steps=100000):
  
    coins = []

    for i in range(n):
        position = populate_coins(coins, R, coin_radius, max_depth=5000)

        coins.append(position)

    x, y = 0.0, 0.0
    dx, dy = scatter()
    
    t_total = 0.0
    
    for step in range(max_steps):
        current_r = np.sqrt(x**2 + y**2)
        
        if current_r >= R:
            dx, dy = reflect(R, x, y)
        
        
        b,disc = discriminant(x,y,dx,dy,R)
        t_boundary = (-b + np.sqrt(disc)) / (2)
        

        t_coin = float('inf')
        hit_idx = -1
        
        for i, (sx, sy) in enumerate(coins):
            rx, ry = x - sx, y - sy
            b_q,disc_q = discriminant(rx,ry,dx,dy,coin_radius)
            
            if disc_q >= 0:
                t1 = (-b_q - np.sqrt(disc_q)) / 2
                if 1e-10 < t1 < t_coin:
                    t_coin = t1
                    hit_idx = i
        
        t_step = min(t_boundary, t_coin)
        
        x += dx * t_step
        y += dy * t_step
        t_total += t_step / v
        
        if hit_idx != -1:
            return t_total
        
        if t_step == t_boundary:
            dx, dy = reflect(R, x, y)
    
    return -100000000  # In case max_steps reached

def simulate_particle_to_wall(R, n, coin_radius, v, max_steps=100000):
#Large coin radius graphs no longer diverge from estimations and can be fairly well curve fitted

    coins = []

    for i in range(n):
        position = populate_coins(coins, R, coin_radius, max_depth=50000)

        coins.append(position)

    x, y = 0.0, 0.0
    dx, dy = scatter()
    
    t_total = 0.0
    
    for step in range(max_steps):
        current_r = np.sqrt(x**2 + y**2)
        
        if current_r >= R:
            dist_to_boundary = R - current_r
            t_total += dist_to_boundary / v 
            return t_total
        
        b,disc = discriminant(x,y,dx,dy,R)
        if disc < 0:
            t_boundary = float('inf')
        else:
            t_boundary = (-b + np.sqrt(disc)) / (2)
            if t_boundary < 0:
                t_boundary = (-b - np.sqrt(disc)) / (2)
            if t_boundary < 0:
                t_boundary = float('inf')
        
        t_coin = float('inf')
        hit_coin_idx = -1
        
        for i, (sx, sy) in enumerate(coins):
            rx, ry = x - sx, y - sy
            b_q,disc_q = discriminant(rx,ry,dx,dy,coin_radius)
            
            if disc_q >= 0:
                t1 = (-b_q - np.sqrt(disc_q)) / 2
                if 1e-10 < t1 < t_coin:
                    t_coin = t1
                    hit_coin_idx = i
        
        t_step = min(t_boundary, t_coin)
        
        x += dx * t_step
        y += dy * t_step
        t_total += t_step / v
        
        if t_step == t_boundary:
            return t_total
        
        if hit_coin_idx != -1:
            dx, dy = scatter()
    
    return -100000000  # In case max_steps reached

def simulate_coin_decay(R, n, coin_radius, v, max_steps=100000):
  
    coins = []

    for i in range(n):
        position = populate_coins(coins, R, coin_radius, max_depth=5000)

        coins.append(position)

    remaining_coins = coins.copy()  # Start with all coins
    coin_indices = list(range(n))   # Track original indices
    
    x, y = 0.0, 0.0
    dx, dy = scatter()
    
    t_total = 0.0
    coins_removed = 0
    t_total_array = []
    t_total_array.append(t_total)

    for step in range(max_steps):
        current_r = np.sqrt(x**2 + y**2)
        
        if not remaining_coins:
            print(f"All {n} coins removed in time {t_total:.3f}")
            return t_total_array
        
        if current_r >= R:
            dx, dy = reflect(R, x, y)
        
        b,disc = discriminant(x,y,dx,dy,R)
        t_boundary = (-b + np.sqrt(disc)) / (2)

        t_coin = float('inf')
        hit_idx = -1
        
        for i, (sx, sy) in enumerate(remaining_coins):
            rx, ry = x - sx, y - sy
            b_q,disc_q = discriminant(rx,ry,dx,dy,coin_radius)
            
            if disc_q >= 0:
                t1 = (-b_q - np.sqrt(disc_q)) / 2
                if 1e-10 < t1 < t_coin:
                    t_coin = t1
                    hit_idx = i
        
        t_step = min(t_boundary, t_coin)
        
        x += dx * t_step
        y += dy * t_step
        t_total += t_step / v
        
        if hit_idx != -1 and t_step == t_coin:

            coins_removed += 1
            t_total_array.append(t_total)
            print(f"Step {step}: Collected coin {coin_indices[hit_idx]} (total: {coins_removed}/{n})")
            
            # Remove coin
            removed_coin = remaining_coins.pop(hit_idx)
            removed_index = coin_indices.pop(hit_idx)
            
            dx, dy = scatter()
            continue
        
        if t_step == t_boundary:
            dx, dy = reflect(R, x, y)
    
    print(f"Warning: Max steps reached. {len(remaining_coins)}/{n} coins remaining")
    print(f"Time elapsed: {t_total:.3f}")
    
    return t_total_array

#============================================================== Simulation Run Methods ====================================================================================

def run_simulation_to_first_collision(R, coin_radius, v, n, num_trials):
    results = []
    error = []
    
    for t in num_trials:
        print(f"Simulating Trials={t}...")
        times = []
        for trial in range(t):
            x = simulate_particle_to_first_collision(R, n, coin_radius, v)
            times.append(x)
        
        mean_time = np.mean(times)
        std_time = np.percentile(times,95)
        results.append(mean_time)
        error.append(std_time)
        
        print(f"  n={n}: mean time = {mean_time:.3f} ± {std_time:.3f}")
    
    return results, error

def run_simulation_to_wall(R, coin_radius, v, n_values, num_trials):
    results = []
    error = []
    
    for n in n_values:
        print(f"Simulating n={n}...")
        times = []
        for trial in range(num_trials):
            t = simulate_particle_to_wall(R, n, coin_radius, v)
            times.append(t)
        
        mean_time = np.mean(times)
        std_time = np.std(times)
        results.append(mean_time)
        error.append(std_time)
        
        print(f"  n={n}: mean time = {mean_time:.3f} ± {std_time:.3f}")
    
    return results, error

def run_simulation_coin_decay(R, coin_radius, v, n, num_trials):
    results = []
    error = []
    removed_coins = np.arange(0,n+1,1)


    for trial in range(num_trials):
        times = simulate_coin_decay(R, n, coin_radius, v)
        print(times)
        results.append(times)
        error.append(times)

    averaged_results = np.mean(results, axis=0)
    averaged_error = np.std(error, axis=0)
    
    print(averaged_results)
    print(averaged_error)
    print(removed_coins)
        
    return averaged_results, averaged_error, removed_coins


#============================================================== Equation for best fit for problem 2 =======================================================================

def optimal_function(n, J, K, R, v, coin_radius):

    n = np.asarray(n, dtype=float)
    
    ballistic_limit = R / v  # scalar
    
    # Ensure element-wise division for diffusive_limit
    diffusive_limit = (coin_radius * n) / (np.pi * v)  # shape (m,)
    
    # Ensure n**J and n**K work element-wise
    ballistic_scaling = -(np.pi * R) / (coin_radius * (n**J))  # shape (m,)
    diffusive_scaling = -(np.pi * R) / (coin_radius * (n**K))  # shape (m,)
    
    # Element-wise operations
    term1 = ballistic_limit * (1 - np.exp(ballistic_scaling))  # shape (m,)
    term2 = diffusive_limit * (1 + np.exp(diffusive_scaling))  # shape (m,)
    
    return term1 + term2
    
#============================================================== Equation for problem 3 ====================================================================================

def equation_for_decay(R, coin_radius, v, n):
    removed_coins = np.arange(0,n+1,0.1)
    times = []
    for c in removed_coins:
        t = (np.log(n-c)-np.log(n))*(np.pi * R**2)/(-(2*v*coin_radius))
        times.append(t)
    return times


#============================================================== Plots =====================================================================================================

def plot_sim_to_first_collision(results, error, R, coin_radius, v, n_values, trials):

    expected_time = (np.pi * R**2)/(2*v*coin_radius*n_values)
    expected_time_line = np.ones_like(trials) * expected_time

    plt.figure(figsize=(10, 6))
    plt.errorbar(trials, results, yerr=0,fmt='o', capsize=5, label='Simulation mean', color='black')
    plt.plot(trials, expected_time_line, 'r--', label='Expected Time to first collision', linewidth=3)
    plt.xlabel('Number of trials')
    plt.ylabel('Mean time to first collision (τ)/seconds')
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.xlim(-1, max(trials) + 5)
    plt.ylim(bottom=0)
    plt.tight_layout()
    plt.savefig("Mean_Time_To_First_Collision")
    plt.show()

    plt.figure(figsize=(10, 6))
    plt.errorbar(trials, results, yerr=error,fmt='o', capsize=5, label='Simulation mean', color='black')
    plt.plot(trials, expected_time_line, 'r--', label='Expected Time to first collision', linewidth=3)
    plt.xlabel('Number of trials')
    plt.ylabel('Mean time to first collision (τ)/seconds')
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.xlim(-1, max(trials) + 5)
    plt.ylim(bottom=0)
    plt.tight_layout()
    plt.savefig("Mean_Time_To_First_Collision_error")
    plt.show()

def plot_sim_to_wall(results, error, R, coin_radius, v, n_values, params):

    ballistic = np.ones_like(n_values, dtype=float) * (R / v)

    diffusion = 2 * coin_radius * n_values / (np.pi * v)

    J_fit,K_fit = params

    best_fit_curve = optimal_function(n_values, J_fit, K_fit, R, v, coin_radius)

    ballistic = np.ones_like(n_values) * ballistic
    diffusion = np.ones_like(n_values) * diffusion

    plt.figure(figsize=(10, 6))

    plt.errorbar(n_values, results, yerr=error,fmt='o', capsize=5, label='Simulation mean ± std', color='black')

    # Theoretical lines
    plt.plot(n_values, ballistic, 'b--', label='Low Density Limit', linewidth=2)
    plt.plot(n_values, diffusion, 'r--', label='High density limit', linewidth=2)
    plt.plot(n_values, best_fit_curve, 'g-', label='Best Fit Equation', linewidth=3)

    plt.xlabel('Number of coins (N)')
    plt.ylabel('Mean time to wall ($T_{\\mathrm{wall}}$)/seconds')
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.xlim(-1, max(n_values) + 5)
    plt.ylim(bottom=0)
    plt.tight_layout()
    plt.savefig("Mean_Time_To_Boundary")
    plt.show()

def plot_sim_coin_decay(results, error, removed_coins, R, coin_radius, v, n):

    estimated_decay = equation_for_decay(R, coin_radius, v, n)
    removed_coins_for_estimate = np.arange(0,n+1,0.1)

    plt.figure(figsize=(10, 6))
    plt.errorbar(removed_coins, results, yerr=error,fmt='o', capsize=5, label='Simulation mean', color='black')
    plt.plot(removed_coins_for_estimate, estimated_decay, 'r--', label='estimated time to reach N removed coins', linewidth=2)
    plt.xlabel('Number Of Coins Removed ($N_{r}$)')
    plt.ylabel('Mean time to reach $N_{r}$ removed coins ($T_{r}$)/seconds')
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.xlim(-1, max(removed_coins) + 5)
    plt.ylim(bottom=0)
    plt.tight_layout()
    plt.savefig("Mean_Time_To_Decay_error")
    plt.show()

    plt.figure(figsize=(10, 6))
    plt.scatter(removed_coins, results, label='Simulation mean', color='black')
    plt.plot(removed_coins_for_estimate, estimated_decay, 'r--', label='estimated time to reach N removed coins', linewidth=2)
    plt.xlabel('Number Of Coins Removed ($N_{r}$)')
    plt.ylabel('Mean time taken to reach $N_{r}$ removed coins (t)/seconds')
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.xlim(-1, max(removed_coins) + 5)
    plt.ylim(bottom=0)
    plt.savefig("Mean_Time_To_Decay")
    plt.show()



# Parameters
R = 100.0  
coin_radius = 1
v = 1.0 
n_values = np.array([0,10,20,30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180])#,240,280])#,320,360])#,400,440,480, 520, 560, 600, 640, 680, 720, 760, 800, 840, 880, 920, 960, 1000])#

n = 150
trials = [1,2,4,8,16,32,64,128,192,256,320,384,448,512]#,640,768,896,1024,1280,1536,1792,2048]#,2560,3072,3582,4096,5120,6144,7168,8192]

results, error = run_simulation_to_first_collision(R, coin_radius, v, n, trials)

plot_sim_to_first_collision(results, error, R, coin_radius, v, n, trials)

results, error = run_simulation_to_wall(R, coin_radius, v, n_values, num_trials=200)

def fitting_function(n, J, K):
    return optimal_function(n, J, K, R, v, coin_radius)

params, covariance = curve_fit(fitting_function, n_values, results)
print(params)

plot_sim_to_wall(results, error, R, coin_radius, v, n_values, params)

results, error, removed_coins = run_simulation_coin_decay(R, coin_radius, v, n, num_trials=100)

plot_sim_coin_decay(results, error, removed_coins, R, coin_radius, v, n)